In [1]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
text = """Многие военные и политические эксперты оценивают нынешнюю обстановку в мире, как крайне опасную и острую, в первую очередь из-за угрозы применения ядерного оружия. За последние десятилетия еще не было столь высокого риска развязывания войны с применением подобного вооружения как сейчас.
Доцент кафедры политического анализа и социально-психологических процессов РЭУ им. Плеханова Александр Перенджиев в колонке для ФАН обратил внимание, что за сложившуюся ситуацию в мире в ответе США и страны Запада. Именно эти государства разрушили равноправную международную систему безопасности. На фоне этого политолог предложил свой метод борьбы с агрессией недружественных государств — через формирование антизападного ядерного союза.
В этот раз я хотел бы поговорить о необходимости создания антизападного ядерного союза с участием России. В настоящее время, по сути, разрушена прежде всего правовая система обеспечения международной безопасности в мире. Мы помним выход Штатов из ПРО (Договор об ограничении систем противоракетной обороны), помним их выходы и из других договоров. Даже Договор о сокращении стратегических наступательных вооружений Россия была вынуждена заморозить. Все по причине агрессивных действий США."""

type_prompt = "Определи, является ли текст: новостью, статьёй, записью в личном блоге, публичным комментарием или личным сообщением. Верни только категорию текста. Если текст не относится ни одной из этих категорий, верни 'другое'."

system_content = "Ты - журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США)."
references_prompt = """Есть ли в тексте ссылки на источники информации? Если они отсутствуют, верни только "Источники: не обнаружены."

Текст:
"""
hatespeech_prompt = """Проверь текст на наличие дискриминирующих, стигматизирующих или провоцирующих ненависть лексики, высказываний или идей (например, трансфобию, гомофобию, национализм, расизм или сексизм).

Текст:
"""

manipulation_prompt = """Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка. Если в тексте нет приёмов манипуляции, верни "Манипуляция: не обнаружена".

Текст:
"""

logic_err_prompt = """Проверь текст (внутри тройной `) на наличие логических ошибок или когнитивных искажений. Если они не обнаружены в тексте, напиши, что "Логические ошибки: не обнаружены". Если они, то сформулируй свой ответ так:
1) Наличие логических ошибок: поочередно перечисли название логических ошибок и для каждой из них кратко (одно предложение) объясни на примере текста, почему это логическая ошибка.
2) Поправка: кратко (в двух предложениях) объясни, почему обнаруженные логические ошибки опасны, и как их избежать.

Например:

Наличие логических ошибок:
1) Атака личности. Уоррен атакует личность Уилла, обвиняя его в ненависти к стране, вместо того чтобы обсуждать его аргументы по поводу военных расходов.
2) Подмена тезиса: Уоррен искажает слова Уилла и потом опровергает своё искажение.

Поправка: Обнаруженные логические ошибки опасны, так как отвлекают от рассмотрения фактов и аргументов, и могут вести к искаженному восприятию ситуации. Чтобы избежать их, важно сосредотачиваться на деле и обсуждении аргументов, а не переходить на личности собеседников.
"""

In [3]:
def process_text(text, temperature=0):
    run_analysis = False
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": f"{type_prompt} Текст: '{text}'"}],
      temperature=temperature,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    text_type = response['choices'][0]['message']['content']
    if text_type.lower() != 'личное сообщение':
        run_analysis = True
    if run_analysis:
        return analyse_text(text, temperature)
    else:
        return {}
    
def analyse_text(text, temperature=0):
    field2prompt = {'manipulation_methods': manipulation_prompt,
               'hatespeech': hatespeech_prompt,
               'references': references_prompt}
    criteria = {field: None for field in field2prompt.keys()}
    for field_name, prompt in field2prompt.items():
        print(f"{prompt} '{text}'")
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "system", "content": system_content},
                    {"role": "user", "content": f"{prompt} '{text}'"}],
          temperature=0,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        answer = response['choices'][0]['message']['content']
        criteria[field_name] = answer
    if criteria['references'] == "Источники: не обнаружены.":
        criteria['references_present'] = 0
    else:
        criteria['references_present'] = 1
    if "Манипуляция: не обнаружена" in criteria['manipulation_methods']:
        criteria['manipulation_present'] = 0
    else:
        criteria['manipulation_present'] = 1
    return criteria

In [4]:
process_text(text)

{'manipulation_methods': '1. Манипуляция: создание чувства опасности и необходимости принятия мер. Автор утверждает, что ситуация в мире крайне опасна и остра из-за угрозы применения ядерного оружия. Это вызывает у читателя страх и требует принятия каких-то действий.\n2. Манипуляция: использование авторитета эксперта. Автор ссылается на доцента кафедры политического анализа и социально-психологических процессов РЭУ им. Плеханова, чтобы подтвердить свою точку зрения и убедить читателя в ее правильности.\n3. Манипуляция: создание образа врага. Автор утверждает, что США и страны Запада разрушили равноправную международную систему безопасности и являются агрессорами. Это создает негативное представление о Западе и подталкивает читателя поддержать идею антизападного ядерного союза.\n4. Манипуляция: предложение альтернативного решения. Автор предлагает создание антизападного ядерного союза с участием России как метода борьбы с агрессией недружественных государств. Это создает впечатление, чт

In [5]:
process_text("Привет! Как дела?")

{}

In [6]:
process_text("\"Ему наплевать\". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии")

{'manipulation_methods': 'Манипуляция: не обнаружена.',
 'hatespeech': 'В данном тексте отсутствуют дискриминирующие, стигматизирующие или провоцирующие ненависть лексики, высказываний или идей.',
 'references': 'Источники: не обнаружены.',
 'references_present': 0,
 'manipulation_present': 0}

In [8]:
process_text("\"Ему наплевать\". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии", 0.29)

{'manipulation_methods': 'Манипуляция: не обнаружена.',
 'hatespeech': 'В данном тексте отсутствуют дискриминирующие, стигматизирующие или провоцирующие ненависть лексики, высказываний или идей.',
 'references': 'Источники: не обнаружены.',
 'references_present': 0,
 'manipulation_present': 0}

In [4]:
manipulation_prompt = """Ты -- журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США). Извлеки все приёмы манипуляции читателем из следующего текста.  Если в тексте нет приёмов манипуляции, подумай ещё раз, если это так, верни "Манипуляция: не обнаружена". Если они есть, верни пронумерованный список приёмов с кратким пояснением.

Текст:"""

In [12]:
process_text("\"Ему наплевать\". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии")

Ты -- журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США). Извлеки все приёмы манипуляции читателем из следующего текста.  Если в тексте нет приёмов манипуляции, подумай ещё раз, если это так, верни "Манипуляция: не обнаружена". Если они есть, верни пронумерованный список приёмов с кратким пояснением.

Текст: '"Ему наплевать". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'
Проверь текст на наличие дискриминирующих, стигматизирующих или провоцирующих ненависть лексики, высказываний или идей (например, трансфобию, гомофобию, национализм, расизм или сексизм).

Текст:
 '"Ему наплевать". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'
Есть ли в тексте ссылки на источники информации? Если они отсутствуют, верни только "Источники: не обнаружены."

Текст:
 '"

{'manipulation_methods': 'Манипуляция: не обнаружена.',
 'hatespeech': 'В данном тексте отсутствуют дискриминирующие, стигматизирующие или провоцирующие ненависть лексики, высказываний или идей.',
 'references': 'Источники: не обнаружены.',
 'references_present': 0,
 'manipulation_present': 0}

In [5]:
text = "\"Ему наплевать\". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии"
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": "Assistant is a large language model trained by OpenAI"},
                {"role": "user", "content": f"{manipulation_prompt} '{text}'"}]
)
response['choices'][0]['message']['content']

'Приемы манипуляции в данном тексте:\n\n1. Использование эмоционально оценочной лексики: фразы "Ему наплевать" и "подставил верного союзника Украины" представляют лидеров в негативном свете и создают негативное впечатление о них.\n2. Обобщение: утверждение о "пристрастии киевского режима к нацистской идеологии" делает общее заявление о всей администрации Украины на основе действий одного человека, что является необоснованным приемом.\n3. Использование некорректной информации: заявление о "пристрастии киевского режима к нацистской идеологии" не подтверждается достоверными источниками или конкретными фактами, поэтому оно может быть использовано для распространения дезинформации.\n4. Релятивизация: отрицание или преувеличение роли и влияния других сторон, таких как Европа и США, может привести к искажению или снижению реального значения их вмешательства в конфликт.\n\nОбращаю ваше внимание, что анализ может быть субъективным и зависит от контекста. В данном списке перечислены возможные пр

In [17]:
text = "\"Ему наплевать\". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии"
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": "Assistant is a large language model trained by OpenAI"},
                {"role": "user", "content": f"{manipulation_prompt} '{text}'"}],
    temperature=0
)
response['choices'][0]['message']['content']

'Манипуляция: не обнаружена.'

In [9]:
text = """Ты - журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США). Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка. Если в тексте нет приёмов манипуляции, верни "Манипуляция: не обнаружена".



Текст: '"Ему наплевать". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'"""
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": "Assistant is a large language model trained by OpenAI"},
                {"role": "user", "content": f"{manipulation_prompt} '{text}'"}]
)
response['choices'][0]['message']['content']

'В данном тексте можно выделить следующие приемы манипуляции:\n\n1. Искажение фактов: Заголовок статьи "Зеленский подставил верного союзника Украины" является субъективным и содержит суждение, которое не подтверждается в тексте. Фактическая информация не предоставлена и не обоснована.\n\n2. Создание негативного имиджа: В дальнейшем тексте утверждается, что киевский режим имеет пристрастие к нацистской идеологии, но не предоставляются конкретные доказательства или ссылки. Это создает негативный имидж Украины и украинских правителей без достаточных оснований.\n\nВ итоге, список приемов манипуляции:\n\n1. Искажение фактов.\n2. Создание негативного имиджа.'

In [11]:
manipulation_prompt

'Ты -- журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США). Извлеки все приёмы манипуляции читателем из следующего текста.  Если в тексте нет приёмов манипуляции, подумай ещё раз, если это так, верни "Манипуляция: не обнаружена". Если они есть, верни пронумерованный список приёмов с кратким пояснением.\n\nТекст:'

In [15]:
text = """Ты - журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США). Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка. Если в тексте нет приёмов манипуляции, верни "Манипуляция: не обнаружена".

Текст: 'Ему наплевать. Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'"""
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": text}]
)
response['choices'][0]['message']['content']

'Манипуляция: не обнаружена.'

In [16]:
text = "\"Ему наплевать\". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии"
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": system_content},
                {"role": "user", "content": f"{manipulation_prompt} '{text}'"}]
)
response['choices'][0]['message']['content']

'1. Использование эмоционально окрашенных слов и фраз: в тексте используется фраза "наплевать", которая создает отрицательное впечатление о Зеленском и его отношении к Украине.\n\n2. Клеймо: в тексте утверждается, что Зеленский подставил верного союзника Украины, что является негативной оценкой его действий.\n\n3. Обобщение: утверждение о пристрастии киевского режима к нацистской идеологии, без конкретных фактов и доказательств, создает общий негативный образ киевского правительства.\n\n4. Непроверяемость информации: в тексте не приводятся ссылки на источники или доказательства заявлений о пристрастии киевского режима к нацистской идеологии.\n\n5. Отсутствие контекста: в тексте не анализируются причины и обстоятельства, в которых произошли эти события, что может искажать полную картину происходящего и создавать негативное впечатление.\n\nМанипуляция: обнаружена.'

In [24]:
prompt = """Ты - журналист и исследователь российских СМИ, работающий в условиях диктатуры и военной цензуры (война между Россией и Украиной, с вмешательством Европы и США). Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка. Если в тексте нет приёмов манипуляции, верни "Манипуляция: не обнаружена".
Текст: '"Ему наплевать". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'"""
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": prompt}]
)
response['choices'][0]['message']['content']

'1. Манипуляция: использование эмоциональной подачи ("наплевать", "верный союзник", "нацистская идеология") для создания негативного образа Зеленского и украинского режима.\n2. Манипуляция: употребление термина "киевский режим" для создания имиджа нестабильности и неправомерности украинской власти.\n3. Манипуляция: утверждение о "пристрастии киевского режима к нацистской идеологии" без конкретных доказательств и фактов, направленное на подрыв авторитета Зеленского и украинского режима.\n4. Манипуляция: использование слова "подставил" для создания образа Зеленского как предательского и недостойного доверия лидера.\n5. Манипуляция: упоминание "вмешательства Европы и США" без дальнейшей развертки данной темы, создающее предположение о негативном воздействии западных стран на ситуацию.'

In [44]:
prompt = """Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка. Только если никакие приёмы не обнаружены в тексте, верни "Манипуляция: не обнаружена".
Текст: '"Ему наплевать". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'"""
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": prompt}]
)
response['choices'][0]['message']['content']

'Манипуляция: не обнаружена.'

In [53]:
prompt = """Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка, если они есть, иначе, если текст не содержит никаких признаков манипуляции, верни Нет приёмов.
Текст: '"Ему наплевать". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'"""
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": prompt}]
)
response['choices'][0]['message']['content']

'1. Заголовок "Ему наплевать". Этот заголовок является эмоциональным и провокационным, цель которого - привлечь внимание читателя и вызвать у него негативное отношение к Зеленскому. Приём используется для создания предвзятого отношения читателя к теме статьи.\n\n2. Утверждение о подставе Зеленским верного союзника Украины. В статье нет конкретных доказательств или фактов, которые могли бы подтвердить это утверждение. Таким образом, используется приём необоснованных обвинений с целью создания негативного образа Зеленского перед читателем.\n\n3. Обвинение киевского режима в пристрастии к нацистской идеологии. Подобное обвинение без доказательств или контекста может вызывать негативное отношение к Украине и её лидерам. Приём используется для дискредитации киевского режима и создания негативного образа перед читателями.'

In [36]:
prompt = """Извлеки все приёмы манипуляции читателем из следующего текста и их краткое обоснование в виде пронумерованного списка. Если в тексте нет приёмов манипуляции, проведи анализ ещё два раза, если приёмы всё равно не найдены, верни "Манипуляция: не обнаружена". 
Текст: '"Ему наплевать". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии'"""
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": prompt}]
)
response['choices'][0]['message']['content']

'Манипуляция: не обнаружена'

In [16]:
text = "\"Ему наплевать\". Зеленский подставил верного союзника Украины. Кремль заявил о пристрастии киевского режима к нацистской идеологии"
response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": "Assistant is a large language model trained by OpenAI"},
                {"role": "system", "content": system_content},
                {"role": "user", "content": f"{manipulation_prompt} '{text}'"}]
)
response['choices'][0]['message']['content']

'Прежде чем я приступлю к анализу текста, хочу отметить, что у меня нет реального доступа к информации о конкретных происшествиях и действиях в условиях вышеописанного сценария. В качестве искусственного интеллекта, я могу анализировать тексты и предлагать общие наблюдения, основанные на широкой базе знаний и опыте, но мои ответы следует рассматривать как гипотетические.\n\nТеперь, относительно текста, можно выделить несколько приемов манипуляции:\n\n1. Заголовок с отрицательной коннотацией: Заголовок "Ему наплевать". Он негативно влияет на оценку Зеленского и создает предубежденность против него до даже прочтения статьи.\n2. Подмена фактов: Текст утверждает, что Зеленский "подставил верного союзника Украины", но не предоставляет необходимой информации или контекста, чтобы это утверждение имело основание. Необъясненное и невыдержанное утверждение может вызывать конфликт восприятия у читателя.\n3. Некатегорическое заявление без подтверждающих фактов: Утверждение "Кремль заявил о пристра